In [2]:
import os 
from time import perf_counter
from typing import Dict
import pickle
import traceback

import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sift import compute_sift

In [3]:
dataset_dir ="C:/Users/admin/Desktop/codespace/python/Structure-from-Motion/dataset/penholder"
output_dir ="C:/Users/admin/Desktop/codespace/python/output_sfm"

In [4]:
if os.path.isdir(dataset_dir) is not True:
    raise NotADirectoryError(f'The Directory {datadir} not found')

if os.path.isdir(output_dir) is not True:
    raise NotADirectoryError(f'The Directory {output_dir} not found')    
    
dataset_dir = os.path.abspath(dataset_dir)    
files = [os.path.join(dataset_dir,file ) for file in os.listdir(dataset_dir)]

print(f'files count {len(files)}')


files count 83


In [5]:
def get_basename(file_path:str) -> str:
    """Extracts Basename of File with out Extension."""
    baseName = os.path.basename(file_path).split(".")[:-1]
    return ".".join(baseName) if isinstance(baseName, list) else baseName

In [6]:
def compute_sift(image_path:str, output_dir:str = output_dir) ->str:
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image,None)
    pickle_kp=list(map(lambda p: (p.pt[0],p.pt[0], p.size, p.angle, p.response, p.octave, p.class_id),keypoints))
    #print(pickle_kp[1], len(pickle_kp))
    features = {
        "keypoint": pickle_kp,
        "descriptor": descriptors,
    }
    base_name = get_basename(image_path)
    pickle_file_path=os.path.join(output_dir,f'{base_name}.pickle')
    with open(pickle_file_path, mode="wb") as pkl_file:
        pickle.dump(features, pkl_file)
    return pickle_file_path    

In [7]:

print(f'Number of Cores {cv2.getNumThreads()}')
print(f'Have Opencl {cv2.ocl.haveOpenCL()}')
cv2.cuda.getCudaEnabledDeviceCount()

Number of Cores 6
Have Opencl True


0

In [8]:
start_time = perf_counter()

#for file in tqdm(files, desc="Extracting Sift Feature ..."):
#    pkl_file=compute_sift(file)
    
end_time = perf_counter()
execution_time = end_time - start_time
print('took {0:.8f}s to execute'.format(execution_time))    

took 0.00003070s to execute


In [ ]:
from concurrent.futures import  ProcessPoolExecutor, ThreadPoolExecutor, as_completed

In [ ]:
start_time = perf_counter()
thread=cv2.getNumThreads()
cv2.setNumThreads(0)

args = list(map(lambda x: (x,output_dir), files))

with ProcessPoolExecutor(3)as executator:
    result = [executator.submit(compute_sift, arg) for arg in args]
    
    for f in as_completed(result):
        print(f.result())

end_time = perf_counter()
#cv2.setNumThreads(thread)
print('took {0:.8f}s to execute'.format(end_time - start_time))       

In [9]:
import multiprocess as mp

In [ ]:
start_time = perf_counter()
thread=cv2.getNumThreads()
cv2.setNumThreads(0)

args = list(map(lambda x: (x,output_dir), files))

with mp.Pool(4) as pool:
    pool.map(compute_sift, args)
    
end_time = perf_counter()
cv2.setNumThreads(thread)
print('took {0:.8f}s to execute'.format(end_time - start_time))     